In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import musicbrainzngs as mb
import geopy
import folium

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

mb.set_useragent(app='8gigjamz', version='0.0.1', contact='sabanaqvi97@gmail.com')

In [2]:
CLIENT_ID = '4fa8860dd8be41558c056e07f369311e'
CLIENT_SECRET = 'a3e7adf575754dd7aa10d54364e4dc90'
REDIRECT_URL = 'http://localhost:8080/callback'

JAMZ_PLAYLIST_ID = '3KPM1xpY0f4XIXV99DI4SO'

In [3]:
scope = ['playlist-read-private', 'user-top-read']

auth_manager = SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET,
                            redirect_uri=REDIRECT_URL, scope=scope)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# A clunky way to get the top 99? Maybe...
top_artists_1 = sp.current_user_top_artists(limit=49, time_range='long_term')
top_artists_2 = sp.current_user_top_artists(offset=49, limit=50, time_range='long_term')
top_artists = pd.concat([pd.json_normalize(top_artists_1['items'])['name'], 
                         pd.json_normalize(top_artists_2['items'])['name']], 
                         ignore_index=True)

In [5]:
top_artists_origins = []
for artist in top_artists:
    try:
        origin = mb.search_artists(query=artist, limit=2)
        top_artists_origins.append(f"{origin['artist-list'][0]['begin-area']['name']}, {origin['artist-list'][0]['area']['name']}")
    except KeyError:
        top_artists_origins.append('Null')
        continue

In [6]:
top_artists_origins = pd.Series(top_artists_origins)
top_artists_df = pd.concat([top_artists, top_artists_origins], axis=1)
top_artists_df.columns = ['artist', 'origin']
top_artists_df.set_index('artist', inplace=True)
top_artists_df

,origin
artist,
Glass Animals,"Oxford, United Kingdom"
The Strokes,"New York, United States"
Stromae,"Brussels, Belgium"
Big Baby Tape,"Moscow, Moscow"
Arctic Monkeys,"High Green, United Kingdom"
Led Zeppelin,"London, United Kingdom"
SUNMI,"Iksan, South Korea"
Kid Cudi,"Cleveland, United States"
Gorillaz,"Essex, England"


In [7]:
# Correct errors, fill gaps, and establish consistency
top_artists_df['origin'] = top_artists_df['origin'].str.replace('England', 'United Kingdom')
top_artists_df.loc['Big Baby Tape', 'origin'] = 'Moscow, Russia'
top_artists_df.loc['Gorillaz', 'origin'] = 'London, United Kingdom'
top_artists_df.loc['Saba', 'origin'] = 'Chicago, United States'
# Nobody associates Kanye with Atlanta...
top_artists_df.loc['Kanye West', 'origin'] = 'Chicago, United States'
top_artists_df.loc['Steve Lacy', 'origin'] = 'Compton, United States'
top_artists_df.loc['A.R. Rahman', 'origin'] = 'Madras, India'
top_artists_df.loc['Lil Nas X', 'origin'] = 'Lithia Springs, United States'
top_artists_df.loc['Mohammed Rafi', 'origin'] = 'Amritsar, India'
# Nobody associates Elliott Smith with Nebraska...
top_artists_df.loc['Elliott Smith', 'origin'] = 'Portland, United States'
top_artists_df.loc['Miguel', 'origin'] = 'Los Angeles, United States'
top_artists_df.loc['Jungle', 'origin'] = 'London, United Kingdom'
top_artists_df.loc['Rihanna', 'origin'] = 'Saint Michael, Barbados'
top_artists_df.loc['Strings', 'origin'] = 'Karachi, Pakistan'
top_artists_df.loc['Eminem', 'origin'] = 'Detroit, Michigan'
# Since geopy picks the incorrect Albany, NY by default
top_artists_df.loc['Ray Charles', 'origin'] = 'Albany, Georgia, United States'
top_artists_df.loc['Kali Uchis', 'origin'] = 'Alexandria, United States'
top_artists_df

,origin
artist,
Glass Animals,"Oxford, United Kingdom"
The Strokes,"New York, United States"
Stromae,"Brussels, Belgium"
Big Baby Tape,"Moscow, Russia"
Arctic Monkeys,"High Green, United Kingdom"
Led Zeppelin,"London, United Kingdom"
SUNMI,"Iksan, South Korea"
Kid Cudi,"Cleveland, United States"
Gorillaz,"London, United Kingdom"


In [9]:
geolocator = geopy.Nominatim(user_agent='example')

def get_latitude(address):
    location = geolocator.geocode(address)
    if location:
        return location.latitude
    else:
        return None
    
    
def get_longitude(address):
    location = geolocator.geocode(address)
    if location:
        return location.longitude
    else:
        return None
    
top_artists_df['latitude'] = top_artists_df['origin'].apply(get_latitude)
top_artists_df['longitude'] = top_artists_df['origin'].apply(get_longitude)

In [10]:
top_artists_df.head(5)

,origin,latitude,longitude
artist,,,
Glass Animals,"Oxford, United Kingdom",51.752013,-1.257850
The Strokes,"New York, United States",40.712728,-74.006015
Stromae,"Brussels, Belgium",50.846557,4.351697
Big Baby Tape,"Moscow, Russia",55.750446,37.617494
Arctic Monkeys,"High Green, United Kingdom",53.474215,-1.493166


In [11]:
# Map of musicians' origins -- hover over markers to see artist!   
top_artists_df.reset_index(inplace=True)

music_map = folium.Map(location=[36.7783, -119.4179], 
                            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
                            attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
                            zoom_start=3)


for i, row in top_artists_df.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), tooltip=row.artist, radius=2, opacity=1, weight=2, color='blue', fill='blue').add_to(music_map)


music_map